In [ ]:
import re

import kagglehub
import keras
import keras_hub
import keras_tuner
import pandas as pd
import pkuseg
import tensorflow as tf
from datasets import Dataset
from pypinyin import lazy_pinyin
from keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Input,
    Concatenate,
    Attention,
    StringLookup,
    TextVectorization,
)
from keras.models import Model


path = kagglehub.dataset_download("noxmoon/chinese-official-daily-news-since-2016")

print("Path to dataset files:", path)

/Users/perrineqhn/Desktop/M2/S2/Méthode en Apprentissage Automatique/tf_metal_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/perrineqhn/.cache/kagglehub/datasets/noxmoon/chinese-official-daily-news-since-2016/versions/1


# Création du corpus

In [3]:
dataset = pd.read_csv(path+"/chinese_news.csv")
# Print dataset information
print("Dataset information:")
print(dataset.info())
# Print dataset head
print("Dataset head:")
print(dataset.head())

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20738 entries, 0 to 20737
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      20738 non-null  object
 1   tag       20738 non-null  object
 2   headline  20738 non-null  object
 3   content   20631 non-null  object
dtypes: object(4)
memory usage: 648.2+ KB
None
Dataset head:
         date   tag                                           headline  \
0  2016-01-01  详细全文  陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...   
1  2016-01-01  详细全文                             中央军委印发《关于深化国防和军队改革的意见》   
2  2016-01-01  详细全文                           《习近平关于严明党的纪律和规矩论述摘编》出版发行   
3  2016-01-01  详细全文                                 以实际行动向党中央看齐 向高标准努力   
4  2016-01-01  详细全文                                 【年终特稿】关键之年 改革挺进深水区   

                                             content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...  
1  经中央军委主席习近平批准，中央军委近日印发了《关

In [4]:
# Prétraitement de content (suppression des caractères non chinois, normalisation des espaces)
def clean_content(text):
    if not isinstance(text, str):
        return ""
    
    # Garder les caractères chinois et ponctuation chinoise
    text = re.sub(r"[^\u4e00-\u9fff\u3000-\u303F\uff00-\uffef]", "", text)
    
    # Normaliser les espaces (rare, mais au cas où)
    text = text.replace(" ", "").strip()

    return text

# Remplacer les valeurs manquantes par une chaîne vide
dataset["content"] = dataset["content"].fillna("")

# Appliquer le prétraitement à la colonne 'content'
dataset['cleaned_content'] = dataset['content'].apply(clean_content)

# Filtrer les lignes où 'cleaned_content' est vide
dataset = dataset[dataset["cleaned_content"].str.strip() != ""].reset_index(drop=True)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after preprocessing:")
print(dataset[['content', 'cleaned_content']].head())

seg = pkuseg.pkuseg()
dataset["tokens"] = dataset["cleaned_content"].apply(lambda x: seg.cut(x))

# Aperçu
print(dataset["tokens"].head())

Dataset after preprocessing:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                                     cleaned_content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会年月日在...  
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。《意见》强调，...  
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...  
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...  
4  刚刚过去的年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，集中发力...  
0    [中国, 人民, 解放军, 陆军, 领导, 机构, 、, 中国, 人民, 解放军, 火箭军,...
1    [经, 中央军委, 主席, 习近平, 批准, ，, 中央军委, 近日, 印发, 了, 《, ...
2    [由, 中共中央, 纪律, 检查, 委员会, 、, 中共中央, 文献, 研究室, 编辑, 的...
3    [广大, 党员, 干部, 正在, 积极, 学习, 习近平, 总书记, 在, 中央, 政治局,...
4    [刚刚, 过去, 的, 年, ，, 是, 全面, 深化, 改革, 的, 关键, 之, 年, ...
Name: tokens, dtype: object


In [5]:
# convert the content column to pinyin
t9_map = {
    "@": "1", ".": "1", ":": "1",
    "a": "2", "b": "2", "c": "2",
    "d": "3", "e": "3", "f": "3",
    "g": "4", "h": "4", "i": "4",
    "j": "5", "k": "5", "l": "5",
    "m": "6", "n": "6", "o": "6",
    "p": "7", "q": "7", "r": "7", "s": "7",
    "t": "8", "u": "8", "v": "8",
    "w": "9", "x": "9", "y": "9", "z": "9",
    "1": "1", "2": "2", "3": "3", "4": "4",
    "5": "5", "6": "6", "7": "7", "8": "8",
    "9": "9", "0": "0", " ": "0",
    "。":"。", "，":"，", "？":"？", "！":"！",
}

# Fonction pour convertir une chaîne de caractères en code T9
def pinyin_to_t9(text):
    t9_code = ""
    if pd.isna(text):
        return ""
    for char in text.lower():
        t9_code += t9_map.get(char, char)  # Conserver les caractères non mappés
    return t9_code

def validate_t9(t9_code):
    # Vérifie que le code T9 est numérique (ou vide pour ponctuation)
    return bool(re.match(r'^[0-9]+$', t9_code)) or t9_code in {"。", "，", "？", "！"}

def generer_sequence_contextuelle(row):
    tokens = row["tokens"]
    sequence = []
    for token in tokens:
        if not isinstance(token, str) or not re.search(r'[\u4e00-\u9fff]', token):
            continue
        for char, py in zip(token, lazy_pinyin(token)):
            t9 = pinyin_to_t9(py)
            if validate_t9(t9):  # Vérifier que le T9 est valide
                sequence.append(f"{char}|{py}|{t9}")
    return ' '.join(sequence)

dataset["char_pinyin_t9_sequence"] = dataset.apply(generer_sequence_contextuelle, axis=1)

# Filtrer les lignes où 'char_pinyin_t9_sequence' est vide
dataset = dataset[dataset["char_pinyin_t9_sequence"].str.strip() != ""].reset_index(drop=True)

# Sauvegarder le fichier
dataset[["char_pinyin_t9_sequence"]].to_csv("sequences_char_pinyin_t9.csv", index=False)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after generating sequences:")
print(dataset[['content', 'char_pinyin_t9_sequence']].head())

Dataset after generating sequences:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                             char_pinyin_t9_sequence  
0  中|zhong|94664 国|guo|486 人|ren|736 民|min|646 解|...  
1  经|jing|5464 中|zhong|94664 央|yang|9264 军|jun|58...  
2  由|you|968 中|zhong|94664 共|gong|4664 中|zhong|94...  
3  广|guang|48264 大|da|32 党|dang|3264 员|yuan|9826 ...  
4  刚|gang|4264 刚|gang|4264 过|guo|486 去|qu|78 的|de...  


# Création du dataset pour le modèle

In [6]:
# Modifier la génération des séquences pour inclure le contexte
input_t9_sequences = []
context_char_sequences = []
target_char_sequences = []
MAX_CONTEXT_LENGTH = 50  # Longueur max du contexte
MAX_T9_LENGTH = 50      # Longueur max de la séquence T9
MAX_TARGET_LENGTH = 50  # Longueur max de la séquence cible

for seq in dataset["char_pinyin_t9_sequence"]:
    triplets = seq.strip().split(" ")
    t9_seq = []
    char_seq = []
    
    for triplet in triplets:
        parts = triplet.split("|")
        if len(parts) == 3:
            char, _, t9 = parts
            if validate_t9(t9):
                char_seq.append(char)
                t9_seq.append(t9)
    
    # Créer des paires contexte-T9-cible
    for i in range(1, len(t9_seq)):
        # Contexte : caractères avant la position i
        context = char_seq[:i][-MAX_CONTEXT_LENGTH:]
        # Entrée T9 : codes T9 à partir de i
        t9_input = t9_seq[i:i+MAX_T9_LENGTH]
        # Cible : caractères à partir de i
        target = char_seq[i:i+MAX_TARGET_LENGTH]
        
        if context and t9_input and target:
            context_char_sequences.append("".join(context))
            input_t9_sequences.append(" ".join(t9_input))
            target_char_sequences.append("".join(target))

# Créer un DataFrame
df_sequences = pd.DataFrame({
    "context_char_sequence": context_char_sequences,
    "input_t9_sequence": input_t9_sequences,
    "target_char_sequence": target_char_sequences
})

# Filtrer les séquences vides
df_sequences = df_sequences[
    (df_sequences["context_char_sequence"].str.strip() != "") &
    (df_sequences["input_t9_sequence"].str.strip() != "") &
    (df_sequences["target_char_sequence"].str.strip() != "")
]

In [7]:
print("DataFrame sequences:")
print(df_sequences.head())

DataFrame sequences:
  context_char_sequence                                  input_t9_sequence  \
0                     中  486 736 646 543 3264 586 58 586 5464 326 54 46...   
1                    中国  736 646 543 3264 586 58 586 5464 326 54 468 94...   
2                   中国人  646 543 3264 586 58 586 5464 326 54 468 94664 ...   
3                  中国人民  543 3264 586 58 586 5464 326 54 468 94664 486 ...   
4                 中国人民解  3264 586 58 586 5464 326 54 468 94664 486 736 ...   

                                target_char_sequence  
0  国人民解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一大...  
1  人民解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一大楼...  
2  民解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一大楼隆...  
3  解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一大楼隆重...  
4  放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一大楼隆重举...  


In [8]:
# # Utiliser tf.data.Dataset
# tf_dataset = tf.data.Dataset.from_tensor_slices((input_t9_sequences, target_char_sequences)).prefetch(tf.data.AUTOTUNE)
# tf_dataset.take(1).get_single_element()

## Encoder les données pour Keras

In [ ]:
# TextVectorization pour le contexte, T9 et cible
context_tv = keras.layers.TextVectorization(
    output_mode='int',
    split='character',
    standardize=None,
    ragged=True
)

input_t9_tv = keras.layers.TextVectorization(
    output_mode='int',
    split='whitespace',
    standardize=None,
    ragged=True
)

target_tv = keras.layers.TextVectorization(
    output_mode='int',
    split='character',
    standardize=None,
    ragged=True
)

# Créer tf.data.Dataset
tf_dataset = tf.data.Dataset.from_tensor_slices(
    (
        df_sequences["context_char_sequence"].values,
        df_sequences["input_t9_sequence"].values,
        df_sequences["target_char_sequence"].values
    )
).prefetch(tf.data.AUTOTUNE)

# Adapter les vectoriseurs
context_ds = tf_dataset.map(lambda ctx, t9, tgt: ctx)
t9_ds = tf_dataset.map(lambda ctx, t9, tgt: t9)
target_ds = tf_dataset.map(lambda ctx, t9, tgt: tgt)
context_tv.adapt(context_ds)
input_t9_tv.adapt(t9_ds)
target_tv.adapt(target_ds)

In [ ]:
# Transformer le dataset
@tf.function
def transform_ds(ctx, t9, tgt):
    vectorized_ctx = context_tv(ctx)
    vectorized_t9 = input_t9_tv(t9)
    vectorized_tgt = target_tv(tgt)
    return (vectorized_ctx, vectorized_t9), vectorized_tgt

transformed_tf_dataset = tf_dataset.map(transform_ds, num_parallel_calls=tf.data.AUTOTUNE)

In [24]:
transformed_tf_dataset.padded_batch(3).take(1).get_single_element()

(<tf.Tensor: shape=(3, 100), dtype=int64, numpy=
 array([[ 10,   5,  27,  78,  38,  23,  43,  21,  43,  28,  31,   3,  90,
          10,   5,  27,  78,  38,  23,  43,   5,  13,  43,  10,   5,  27,
          78,  38,  23,  43,  14, 116,  15,  24,  26,  76,  59,   3,   9,
          17,  36,  54,   8,  16,  97,   2,   9, 149, 108,  10,  21,  20,
          10,  22,  10,  47,  56,  52,   3,   5,  44,  46,   2,  10,  47,
          43,  18,  46,   2,   2,  12,  51,  30,  21,  43,   5,  13,  43,
          14, 116,  15,  24,  26,  76,  74,   7,  43,   8, 103,  15,  41,
          60,  86,  39,  23,  10,  47,  11,  10,  47],
        [ 28,  10,  47,  43,  18,  46,   2,   2,  12,  51,   8, 157,  10,
          47,  43,  18,  12,   8,  34,   9,  25,  58,   7,  89,  68,   5,
          23,  11,  43,  76,  66,  11,   6,   2,  13,   2,  13,  42,  77,
          23,   6,   4,  97,   9,   2,  50,  23,  10,  47,  10,  47,  43,
          18,  11,   2,  46,   2,  18,  88,   4,  14,  42,  43,  72,  39,
        

## Split train-valid-test

In [25]:
c = transformed_tf_dataset.reduce(0, lambda x,_:x+1).numpy()

shuffled_ds = transformed_tf_dataset.shuffle(buffer_size=c, seed=42)

train_size = c * 80 // 100
test_size = c * 10 // 100
val_size = c - train_size - test_size

ds_train = shuffled_ds.take(train_size).prefetch(tf.data.AUTOTUNE)
ds_val = shuffled_ds.skip(train_size).take(val_size).prefetch(tf.data.AUTOTUNE)
ds_test = shuffled_ds.skip(train_size+val_size).take(test_size).prefetch(tf.data.AUTOTUNE)

print("Taille du train :", ds_train.cardinality().numpy())
print("Taille du validation :", ds_val.cardinality().numpy())
print("Taille du test :", ds_test.cardinality().numpy())

Taille du train : 16503
Taille du validation : 2064
Taille du test : 2062


# Modèle

Sogou T9 est une méthode d’entrée intelligente qui :

- Prend des séquences numériques (ex. : "94664 486" pour "zhong guo").
- Génère des séquences de caractères chinois (ex. : "中国").
- Utilise le contexte (mots précédents) pour désambiguïser les prédictions.
- Est optimisé pour la vitesse et la précision, souvent avec des modèles entraînés sur de vastes corpus.

Pour reproduire cela, il faut utiliser un modèle seq2seq avec un encodeur-décodeur (2 entrées) :

- Encodeur : Lit la séquence T9 et la compresse en une représentation contextuelle.
- Décodeur : Génère la séquence de caractères chinois à partir de cette représentation.

[Functional API](https://keras.io/guides/functional_api/)

Schéma conceptuel :

[Contexte sinogrammes] -> [LSTM Encodeur Contexte] -> [Représentation contexte] </br>
[T9 entrée]           -> [LSTM Encodeur T9]       -> [Représentation T9] </br>
[Représentations contexte + T9] -> [Attention] -> [LSTM Décodeur] -> [Sortie caractères]

In [ ]:
def build_model():
    # Encodeur 1 : Contexte en sinogrammes
    context_input = Input(shape=(None,), dtype=tf.int32, name="context_input")
    context_embedding = Embedding(
        input_dim=context_tv.vocabulary_size(),
        output_dim=128,
        mask_zero=True
    )(context_input)
    context_lstm = LSTM(256, return_sequences=True, return_state=True)
    context_lstm_out, context_state_h, context_state_c = context_lstm(context_embedding)

    # Encodeur 2 : Entrée T9
    t9_input = Input(shape=(None,), dtype=tf.int32, name="t9_input")
    t9_embedding = Embedding(
        input_dim=input_t9_tv.vocabulary_size(),
        output_dim=128,
        mask_zero=True
    )(t9_input)
    t9_lstm = LSTM(256, return_sequences=True, return_state=True)
    t9_lstm_out, t9_state_h, t9_state_c = t9_lstm(t9_embedding)

    # Combiner les états pour initialiser le décodeur
    combined_state_h = Dense(256, activation='tanh')(Concatenate()([context_state_h, t9_state_h]))
    combined_state_c = Dense(256, activation='tanh')(Concatenate()([context_state_c, t9_state_c]))

    # Décodeur
    decoder_input = Input(shape=(None,), dtype=tf.int32, name="decoder_input")
    decoder_embedding = Embedding(
        input_dim=target_tv.vocabulary_size(),
        output_dim=128,
        mask_zero=True
    )(decoder_input)
    decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
    decoder_lstm_out, _, _ = decoder_lstm(
        decoder_embedding,
        initial_state=[combined_state_h, combined_state_c]
    )

    # Attention pour lier le contexte et T9 au décodeur
    attention = Attention()([decoder_lstm_out, Concatenate()([context_lstm_out, t9_lstm_out])])
    decoder_combined = Concatenate()([decoder_lstm_out, attention])

    # Couches de prédiction
    dense = Dense(512, activation='relu')(decoder_combined)
    output = Dense(target_tv.vocabulary_size(), activation='softmax')(dense)

    # Modèle complet
    model = Model(
        inputs=[context_input, t9_input, decoder_input],
        outputs=output
    )
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Modèle
model = build_model()
model.utils.plot_model(model, show_shapes=True, show_layer_names=True)

# Génération

In [ ]:
def generate_text(context, t9_sequence, max_length=50):
    # Vectoriser les entrées
    context_vector = context_tv([context])
    t9_vector = input_t9_tv([t9_sequence])
    
    # Initialiser la séquence générée
    generated = [target_tv.get_vocabulary().index("<START>")]
    output = []
    
    for _ in range(max_length):
        decoder_input = tf.constant([generated], dtype=tf.int32)
        pred = model.predict([context_vector, t9_vector, decoder_input], verbose=0)
        next_char_idx = tf.argmax(pred[0, -1, :]).numpy()
        next_char = target_tv.get_vocabulary()[next_char_idx]
        
        if next_char == "<END>":
            break
        
        output.append(next_char)
        generated.append(next_char_idx)
    
    return "".join(output)

# Exemple d'utilisation
context = "中国人民"
t9_input = "543 3264"  # Pour "jie fang"
result = generate_text(context, t9_input)
print(f"Contexte: {context}, T9: {t9_input} -> Résultat: {result}")